# Mobilité - Montréal
## Modélisation du mode de transport des individus
### Rapport final - Équipe 4
#### Samuel Chapleau - Francis Forget - Vincent Labonté - Dominique Piché - Michaël Salmon
#### 23 avril 2019
___

## Table des matières

1. [Objectif](#goal)  
2. [Méthode](#method)
2. [Traitement des données](#data_processing)  
    1. [Chargement des données de l'ensemble d'entraînement](#section_load_data) 
    2. [Nettoyage des données](#section_clean_data)  
3. [Analyse exploratoire](#section_exploratory_analysis)  
    - ##### Mode de transport  
        1. [Mode de transport](#exploratory_analysis_mode_of_transport)  
    - ##### Ménage  
        2. [Nombre de véhicules du ménage](#exploratory_analysis_nb_vehicles)  
        3. [Nombre de personnes du ménage](#exploratory_analysis_nb_people)  
        4. [Nombre de véhicules par personne du ménage](#exploratory_analysis_nb_vehicles_per_person)  
        5. [Secteur municipal du ménage](#exploratory_analysis_municipal_sector)  
        6. [Catégorie des revenus du ménage](#exploratory_analysis_revenue)  
        7. [Distance réseau de la station de train la plus proche à partir du domicile du ménage](#exploratory_analysis_train_distance) 
        8. [Distance réseau de la station de métro la plus proche à partir du domicile du ménage](#exploratory_analysis_metro_distance) 
        9. [Distance réseau de la station de train ou de métro la plus proche à partir du domicile du ménage](#exploratory_analysis_distance_tc)   
        10. [Accessibilité du ménage à l’emploi en transport en commun](#exploratory_analysis_public_transport_job_accessibility)  
        11. [Distance réseau jusqu'au centre-ville à partir du domicile du ménage](#exploratory_analysis_downtown_distance)  
    - #### Personne
        12. [Sexe de la personne](#exploratory_analysis_sex)  
        13. [Âge de la personne](#exploratory_analysis_age)  
        14. [Groupe d'âge de la personne](#exploratory_analysis_age_group)  
        15. [Occupation principale de la personne](#exploratory_analysis_occupation)  
        16. [Possession d’un permis de conduire de la personne](#exploratory_analysis_drivers_license)  
    - ##### Déplacement
        17. [Heure de départ du déplacement](#exploratory_analysis_departure_time)  
        18. [Motif du déplacement](#exploratory_analysis_traveling_reason)  
4. [Création des modèles](#model_creation)  
    1. [Validation des modèles](#model_validation)  
    2. [Essais](#model_tries)  
        1. [Variables continues seulement](#model_continuous_variables_only)  
        2. [Variables catégorielles seulement](#model_categorical_variables_only)  
        3. [Variables catégorielles et continues catégorisées](#model_categorical_and_categorized_continuous_variables) 
    3.  [Modèle final](#final_model) 
    4.  [Outils](#model_utils)      
  
5. [Améliorations possibles](#possible_ameliorations)  
    1. [*Feature engineering*](#possible_ameliorations_feature_engineering)
    2. [Mélanges de lois](#possible_ameliorations_mixture)
    3. [Distributions diverses](#possible_ameliorations_different_distributions)
    4. [Autres techniques](#possible_ameliorations_new_solutions)

<a id="goal"></a>
## Objectif
L'objectif de ce rapport consiste à dresser la situation sur la mobilité de la grande région Montréalaise. Nous allons analyser les données de la dernière enquête Origine-Destination effectuée par Transports Québec. Nous souhaitons identifier les facteurs influençant le mode de transport des individus.

<a id="method"></a>
## Méthode
La méthode utilisé afin de réaliser la tâche est la classification bayésienne naïve avec une estimation des paramètres des modèles générés par la méthode du maximum de vraisemblance.

<a id="data_processing"></a>
## Traitement des données

Le traitement des données consiste à importer les paquets nécessaires ainsi qu’à ajouter notre fichier *utils.jl* contenant plusieurs méthodes utilitaires.

In [1]:
using CSV, DataFrames, Gadfly, Distributions, StatsBase, Compose, Random, MLBase;
using Plots: heatmap, ColorGradient;
include("utils.jl");

┌ Info: Recompiling stale cache file /home/chaime/.julia/compiled/v1.0/CSV/HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1190


ArgumentError: ArgumentError: Package MLBase not found in current path:
- Run `import Pkg; Pkg.add("MLBase")` to install the MLBase package.


<a id="section_load_data"></a>
### Chargement des données de l'ensemble d'entraînement

Le fichier *ODtrain.csv* contenant l'ensemble d'entraînement doit être téléchargé depuis Moodle.

In [2]:
train = CSV.read("ODtrain.csv")
first(train,5)

ArgumentError: ArgumentError: "ODtrain.csv" is not a valid file

In [3]:
n = size(train,1)
p = size(train,2)
println("Le fichier d'entraînement contient $n déplacements et $p variables.")

UndefVarError: UndefVarError: train not defined

<a id="section_clean_data"></a>
### Nettoyage des données

Dans ce jeu de données, il n’y a aucune valeur manquante. La première étape consiste à se débarrasser du type complexe des variables permettant d’avoir des valeurs manquantes. La deuxième étape consiste à transformer les variables du problème en variables catégorielles avec des noms significatifs afin de concevoir de plus beaux graphiques. Plus de détails sur la fonction **clean\_train\_data** sont disponibles à la section [Outils](#model_utils).

In [4]:
clean_train_data!(train)
first(train, 5)

UndefVarError: UndefVarError: clean_train_data! not defined

<a id="section_exploratory_analysis"></a>
## Analyse exploratoire
___

L'analyse exploratoire des données est une étape essentielle, car c'est un processus d'inspection, de nettoyage, de transformation et de modélisation des données visant à

- découvrir des informations utiles ;
- alimenter des conclusions ;
- appuyer la prise de décision. 

Cette section documente notre analyse de chaque variable proposée du problème afin d'en conclure une sélection. Nos justifications d'utilisation, de création ou de retrait de variables se feront à l'aide de graphiques, de résultats et d'explications méthodologiques. Ainsi, pour chacune des variables, un graphique de fréquence et un graphique de proportion selon le mode de transport seront fournis afin d'appuyer nos conclusions.

In [5]:
set_default_plot_size(25cm, 15cm)
Gadfly.push_theme(:dark)
dark_theme_without_colorkey = deepcopy(Gadfly.dark_theme)
dark_theme_without_colorkey.key_position = :none
empty_plot = plot(x=[0,1], y=[0,1], Geom.point, Guide.xticks(ticks=nothing), Guide.yticks(ticks=nothing), Guide.xlabel(nothing), Guide.ylabel(nothing), Theme(default_color="white"));

<a id="exploratory_analysis_mode_of_transport"></a>
### Mode de transport
Cette variable correspond au mode de transport emprunté pour se déplacer. Celle-ci correspond à la variable d'intérêt pour identifier la classe d'un individu.

| Indice | Catégorie |
| --- | --- |
| 1 | Automobile |
| 2 | Transport collectif | 
| 3 | Hybride (auto et transport collectif) |
| 4 | Actif (vélo ou marche) |
| 5 | Autre |


#### Nombre de déplacements en fonction du mode de transport

In [6]:
plot(train, x=:D_Mode_str, Geom.histogram,
    Guide.title("Nombre de personnes employant chaque mode de transport"),
    Guide.xlabel("Mode de transport"), Guide.ylabel("Nombre de personnes"))

UndefVarError: UndefVarError: train not defined

L’utilisation de la voiture comme mode de transport est majoritaire par rapport à tous les autres modes de transport. Les autres modes de transport les plus populaires sont le transport collectif, le transport actif et les modes de transport autres. Le mode de transport le moins prisé est le transport hybride.

<a id="exploratory_analysis_nb_vehicles"></a>
### Nombre de véhicules du ménage

Cette variable correspond au nombre de véhicules du ménage (0 à 14). Selon nos observations, nous avons jugé pertinent de regrouper les ménages ayant quatre voitures et plus sous une même catégorie puisque la proportion des modes de transport de ces ménages semblait être similaire.

#### Fréquence et proportion des modes de transport en fonction du nombre de véhicules du ménage

In [7]:
df = histcat(train, :M_AUTO_str, :M_AUTO)
count_plot = plot(df, x = :M_AUTO_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu nombre de véhicules du ménage"), Guide.xlabel("Nombre de véhicules du ménage"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_AUTO_str, :M_AUTO)
proportion_plot = plot(df, x = :M_AUTO_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu nombre de véhicules du ménage"), Guide.xlabel("Nombre de véhicules du ménage"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, nous remarquons que la catégorie comportant le plus grand nombre de ménages est celle composée de deux véhicules. Nous constatons que la majorité des personnes possède un ou deux véhicules. Déjà, il est facilement possible de constater que les ménages n'ayant aucun véhicule sont moins propices à l’utilisation de la voiture.

Dans le graphique de la proportion, il est possible de remarquer que plus un ménage a de voitures, plus la proportion du mode de transport utilisant la voiture est grande. Cette variable sera d’ailleurs utilisée afin de créer la variable « Nombre de véhicules par personne du ménage ». De plus, il semble que plus un ménage a un nombre élevé de voitures, plus ses membres privilégieraient le mode de transport hybride. Nous remarquons également une corrélation inverse entre la proportion de voitures dans le ménage et le nombre de personnes se déplaçant de manière active ou en transport collectif. Finalement, la proportion de personnes utilisant un mode de transport autre est la plus élevée chez les ménages possédant deux véhicules.

<a id="exploratory_analysis_nb_people"></a>
### Nombre de personnes du ménage

Cette variable correspond au nombre de personnes du ménage (1 à 19). Selon nos observations, nous avons jugé pertinent de regrouper le nombre de personnes d’un ménage étant supérieur à sept sous une même catégorie puisque la proportion des modes de transport de ces catégories semblait être similaire.

#### Fréquence et proportion des modes de transport en fonction du nombre de personnes du ménage

In [8]:
df = histcat(train, :M_PERS_str, :M_PERS)
count_plot = plot(df, x = :M_PERS_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu nombre de personnes du ménage"), Guide.xlabel("Nombre de personnes du ménage"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_PERS_str, :M_PERS)
proportion_plot = plot(df, x = :M_PERS_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu nombre de personnes du ménage"), Guide.xlabel("Nombre de personnes du ménage"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, nous pouvons voir que la quantité de répondants habitant dans un ménage de 2 à 5 personnes est considérablement plus élevée, ce qui signifie que les tendances relevées seront plus significatives pour des ménages de cette taille.

Dans le graphique de la proportion, nous observons que la proportion de personnes se déplaçant en voiture croît lorsqu'on passe d’un à deux membres d'un ménage, mais qu'elle décroît ensuite pour chaque personne additionnelle. La proportion de personnes utilisant le transport collectif semble décroître légèrement en fonction du nombre de personnes, tandis que les proportions utilisant des modes de transport actif et autre augmentent (à partir de deux personnes dans le ménage). Il semble fort improbable pour une personne seule dans un ménage d'utiliser un mode de transport autre.

<a id="exploratory_analysis_nb_vehicles_per_person"></a>
### Nombre de véhicules par personne du ménage

Cette variable correspond au nombre de véhicules par personne du ménage. Celle-ci correspond à une variable créée en divisant le nombre de véhicules d’un ménage par le nombre de personnes du ménage. Selon nos observations, nous avons jugé pertinent de regrouper le nombre de véhicules par personne du ménage étant supérieur à sept sous une même catégorie puisque la proportion des modes de transport de ces catégories semblait être similaire.

| Indice | Nombre de véhicules par personne |
| --- | --- |
| 1 | 0 |
| 2 | ]0, 0.25\[ |
| 3 | \[0.25, 0.5[ |
| 4 | \[0.5, 0.75[ |
| 5 | \[0.75, 10[ |
| 6 | >= 1.0 |


#### Fréquence et proportion des modes de transport en fonction du nombre de véhicules par personne du ménage

In [9]:
df = histcat(train, :M_AUTO_per_PERS_str, :M_AUTO_per_PERS)
count_plot = plot(df, x = :M_AUTO_per_PERS_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu nombre de véhicules par personne du ménage"), Guide.xlabel("Nombre de véhicules par personne du ménage"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_AUTO_per_PERS_str, :M_AUTO_per_PERS)
proportion_plot = plot(df, x = :M_AUTO_per_PERS_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu nombre de véhicules par personne du ménage"), Guide.xlabel("Nombre de véhicules par personne du ménage"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, il est le plus commun d'avoir entre 0.25 et 0.75 voiture par personne du ménage, ou d'avoir plus d'une voiture par ménage. Il est aussi possible de remarquer que la grande majorité des personnes se déplaçant en voiture habitent dans des ménages avec soit entre 0.5 et 0.75 voiture par personne ou avec plus d'une voiture par personne. La majorité des utilisateurs de modes de transport autre sont dans des ménages avec entre 0.25 et 0.75 voiture par personne.

Dans le graphique de la proportion, la proportion de personnes employant la voiture semble être en relation directe avec le nombre de véhicules par personne du ménage, tandis que l'utilisation de modes de transport actifs ou collectifs est en relation inverse. Nous remarquons également une première tendance claire pour le mode de transport hybride, qui croît en fonction de l'augmentation du nombre de véhicules par personne. Les modes de transport autres sont privilégiés environ équitablement dans les quatre catégories comprises entre 0 (exclusif) et 1 (exclusif) voiture par personne, indiquant que cette mesure ne pourrait peut-être pas prédire cette catégorie adéquatement.

<a id="exploratory_analysis_municipal_sector"></a>
### Secteur municipal du ménage

Cette variable correspond au secteur municipal du ménage (113 secteurs municipaux). Celle-ci nous permet de comprendre si le secteur municipal du ménage a réellement un impact sur le mode de transport employé.

#### Fréquence des modes de transport en fonction du secteur municipal du ménage

In [10]:
df = histcat(train, :M_DOMSM_str, :M_DOMSM)
count_plot = plot(df, x = :M_DOMSM_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu secteur municipal du ménage"), Guide.xlabel("Secteur municipal du ménage"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_DOMSM_str, :M_DOMSM)
proportion_plot = plot(df, x = :M_DOMSM_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu secteur municipal du ménage"), Guide.xlabel("Secteur municipal du ménage"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, les modes actifs et collectifs sont fréquemment utilisés dans les secteurs entre 101 et 120, tandis que les modes autres sont surtout utilisés dans les secteurs 307 à 663.

Dans le graphique de la proportion, il semble que plus l'identifiant du secteur augmente, plus les gens délaissent le transport actif ou commun au profit de la voiture et du mode autre.

<a id="exploratory_analysis_revenue"></a>
### Catégorie des revenus du ménage

Cette variable correspond aux catégories des revenus du ménage (8 catégories). Celle-ci nous permet de comprendre si le revenu du ménage a réellement un impact sur le mode de transport employé.

| Indice | Revenus du ménage ($)
| --- | --- |
| 1 | < 30 000 |
| 2 | 30 000 - 59 999 |
| 3 | 60 000 - 89 999 |
| 4 | 90 000 - 119 999 |
| 5 | 120 000 - 149 999 |
| 6 | >= 150 000 | 
| 7 | Refus de répondre |
| 8 | Ne sait pas |

#### Fréquence et proportion des modes de transport en fonction de la catégorie des revenus du ménage

In [11]:
df = histcat(train, :M_revenu_str, :M_revenu)
count_plot = plot(df, x = :M_revenu_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la catégorie des revenus du ménage"), Guide.xlabel("Catégorie des revenus du ménage (\$)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_revenu_str, :M_revenu)
proportion_plot = plot(df, x = :M_revenu_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la catégorie des revenus du ménage"), Guide.xlabel("Catégorie des revenus du ménage (\$)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, nous remarquons que la majorité des ménages ont un revenu entre 30 000\\$ et 119 999\\$ par an.

Dans le graphique de la proportion, ce graphique montre clairement que les ménages les moins bien nantis (< 30 000\\$ / an) privilégient les modes de transport collectifs et actifs par rapport à la voiture, et que plus le revenu croît, plus la voiture reprend de la place au profit de ces deux modes.

<a id="exploratory_analysis_train_distance"></a>
### Distance réseau de la station de train la plus proche à partir du domicile du ménage

Cette variable correspond à la distance réseau de la station de train la plus proche à partir du domicile du ménage. Celle-ci nous permet de comprendre si la distance réseau de la station de train la plus proche à partir du domicile du ménage a réellement un impact sur le mode de transport employé.

#### Fréquence et proportion des modes de transport en fonction de la distance réseau de la station de train la plus proche à partir du domicile du ménage

In [12]:
bin_count = 100; x_min = 0; x_max = 50;
count_plot = plot(train, x = :M_dist_train, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de train\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :M_dist_train)
proportion_plot = plot(df, x = :M_dist_train, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de train\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de la distance réseau catégorisée de la station de train la plus proche à partir du domicile du ménage

In [13]:
df = histcat(train, :M_dist_train_str, :M_dist_train)
count_plot = plot(df, x = :M_dist_train_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de train\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_dist_train_str, :M_dist_train)
proportion_plot = plot(df, x = :M_dist_train_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de train\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

L'utilisation des transports collectifs est moins importante lorsque la distance entre le domicile et la station de train la plus proche est de plus de ~10 km. L'automobile est plus prisée au fur et à mesure que la distance augmente.

Les deux graphiques précédents, soit la proportion d'utilisateurs des divers modes de transport en fonction soit de la distance réseau de la station de train ou de cette même distance regroupée en catégories, indiquent qu'il y a une relation directe entre cette distance et le mode de transport choisi. Plus une personne est proche du train, plus elle est susceptible de se déplacer en transport en commun ou de manière active. Plus elle s'en éloigne, plus elle risque d'utiliser la voiture.

<a id="exploratory_analysis_metro_distance"></a>
### Distance réseau de la station de métro la plus proche à partir du domicile du ménage

Cette variable correspond à la distance réseau de la station de métro la plus proche à partir du domicile du ménage. Celle-ci nous permet de comprendre si la distance réseau de la station de métro la plus proche à partir du domicile du ménage a réellement un impact sur le mode de transport employé.

#### Fréquence et proportion des modes de transport en fonction de la distance réseau de la station de métro la plus proche à partir du domicile du ménage

In [14]:
bin_count = 100; x_min = 0; x_max = 50;
count_plot = plot(train, x = :M_dist_metro, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de métro\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de métro\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :M_dist_metro)
proportion_plot = plot(df, x = :M_dist_metro, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de métro\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de métro\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de la distance réseau catégorisée de la station de métro la plus proche à partir du domicile du ménage

In [15]:
df = histcat(train, :M_dist_metro_str, :M_dist_metro)
count_plot = plot(df, x = :M_dist_metro_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de métro\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de métro\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_dist_metro_str, :M_dist_metro)
proportion_plot = plot(df, x = :M_dist_metro_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de métro\nla plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de métro\nla plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

L'utilisation des transports collectifs et actifs est prépondérante lorsque la distance entre le domicile et la station de métro la plus proche est de moins de ~2 km. L'automobile est plus prisée au fur et à mesure que la distance augmente.

Les deux graphiques précédents, soit la proportion d'utilisateurs des divers modes de transport en fonction soit de la distance réseau de la station de métro ou de cette même distance regroupée en catégories, indiquent qu'il y a une relation directe entre cette distance et le mode de transport choisi. Plus une personne est proche du métro, plus elle est susceptible de se déplacer en transport en commun ou de manière active. Plus elle s'en éloigne, plus elle risque d'utiliser la voiture, un transport hybride, ou un mode autre.

<a id="exploratory_analysis_distance_tc"></a>
### Distance réseau de la station de train ou de métro la plus proche à partir du domicile du ménage

Cette variable correspond à la distance réseau de la station de train ou de métro la plus proche à partir du domicile du ménage. Celle-ci correspond à une variable créée en prenant le minimum entre la distance réseau à une station de train ou à une station de métro. Selon nos observations, nous avons jugé pertinent de prendre le minimum entre les deux puisque l’information combinée peut possiblement mener à un meilleur pouvoir prédictif.

#### Fréquence et proportion des modes de transport en fonction de la distance réseau de la station de train ou de métro la plus proche à partir du domicile du ménage

In [16]:
bin_count = 100; x_min = 0; x_max = 40;
count_plot = plot(train, x = :M_dist_tc, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :M_dist_tc)
proportion_plot = plot(df, x = :M_dist_tc, color = :D_Mode_str, y = :Proportion, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage (km)", orientation=:horizontal), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de la distance réseau catégorisée de la station de train ou de métro la plus proche à partir du domicile du ménage

In [17]:
df = histcat(train, :M_dist_tc_str, :M_dist_tc)
count_plot = plot(df, x = :M_dist_tc_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_dist_tc_str, :M_dist_tc)
proportion_plot = plot(df, x = :M_dist_tc_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage"), Guide.xlabel("Distance réseau de la station de train ou\nde métro la plus proche à partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le graphique de la fréquence, la majorité des personnes habitent à moins de 5km d'une station, et forment aussi la grande majorité des utilisateurs de transport collectif.  

Dans le graphique de la proportion, une tendance claire n'est visible que pour les personnes habitant à entre 0 et 15 km de la station la plus proche; par la suite, il devient plus difficile de repérer des corrélations. Pour ces personnes habitant "proche" des stations, plus elles en sont proches, plus elles privilégient le transport en commun et actif. Plus elles en habitent loin, plus elles prennent la voiture. Cette corrélation est à son plus visible entre 0 et 3 km de la station.

Dans les deux graphiques de la distance catégorisée, en regroupant les distances en catégories, les tendances deviennent encore plus claires. Plus la distance à une station de train ou de métro augmente, moins les gens utilisent le transport collectif ou actif, et plus ils privilégient la voiture.

<a id="exploratory_analysis_public_transport_job_accessibility"></a>
### Accessibilité du ménage à l’emploi en transport en commun

L'accessibilité du ménage à l'emploi en transport en commun correspond au nombre d'emplois accessibles à partir d'un ménage en moins de 45 minutes à 8h le matin. Nous avons regroupé cette variable en catégories afin d'en simplifier l'analyse.

#### Fréquence et proportion des modes de transport en fonction de l'accessibilité du ménage à l’emploi en transport en commun

In [18]:
bin_count = 100; x_min = 0; x_max = 200000;
count_plot = plot(train, x = :M_emploiTC, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde l'accessibilité du ménage à l’emploi en transport en commun"), Guide.xlabel("Accessibilité du ménage à l’emploi en transport en commun\n(Nombre d'emplois)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :M_emploiTC)
proportion_plot = plot(df, x = :M_emploiTC, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde l'accessibilité du ménage à l’emploi en transport en commun"), Guide.xlabel("Accessibilité du ménage à l’emploi en transport en commun\n(Nombre d'emplois)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de l'accessibilité catégorisée du ménage à l’emploi en transport en commun

In [19]:
df = histcat(train, :M_emploiTC_str, :M_emploiTC)
count_plot = plot(df, x = :M_emploiTC_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde l'accessibilité du ménage à l’emploi en transport en commun"), Guide.xlabel("Accessibilité du ménage à l’emploi en transport en commun\n(Nombre d'emplois)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_emploiTC_str, :M_emploiTC)
proportion_plot = plot(df, x = :M_emploiTC_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde l'accessibilité du ménage à l’emploi en transport en commun"), Guide.xlabel("Accessibilité du ménage à l’emploi en transport en commun\n(Nombre d'emplois)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Ici, nous pouvons observer que, comme nous nous y attendrions, plus l'accessibilité à l'emploi en transport en commun est grande, plus ce mode de transport est privilégié pour les déplacements, aux dépens de la voiture et du mode autre, et jusqu'à un certain point, le transport hybride. Cependant, il y a également une corrélation positive entre l'accessibilité  et l'utilisation du transport actif.

<a id="exploratory_analysis_downtown_distance"></a>
### Distance réseau jusqu'au centre-ville à partir du domicile du ménage

Cette donnée représente la distance entre un ménage et la Gare Centrale de Montréal, en kilomètres.

#### Fréquence et proportion des modes de transport en fonction de la distance réseau jusqu'au centre-ville à partir du domicile du ménage

In [20]:
bin_count = 100; x_min = 0; x_max = 65;
count_plot = plot(train, x = :M_dist_centre, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau jusqu'au centre-ville\nà partir du domicile du ménage"), Guide.xlabel("Distance réseau jusqu'au centre-ville\nà partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :M_dist_centre)
proportion_plot = plot(df, x = :M_dist_centre, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la distance réseau jusqu'au centre-ville\nà partir du domicile du ménage"), Guide.xlabel("Distance réseau jusqu'au centre-ville\nà partir du domicile du ménage (km)"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de la distance réseau catégorisée jusqu'au centre-ville à partir du domicile du ménage

In [21]:
df = histcat(train, :M_dist_centre_str, :M_dist_centre)
count_plot = plot(df, x = :M_dist_centre_str, y = :Count, color = :D_Mode_str, Geom.bar,
    Guide.title("Fréquence des modes de transport en fonction\nde la distance réseau jusqu'au centre-ville\nà partir du domicile du ménage"),
    Guide.xlabel("Distance réseau jusqu'au centre-ville\nà partir du domicile du ménage (km)"), Guide.ylabel("Fréquence"),
    Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :M_dist_centre_str, :M_dist_centre)
proportion_plot = plot(df, x = :M_dist_centre_str, y = :Proportion, color = :D_Mode_str, Geom.bar,
    Coord.cartesian(ymin = 0, ymax = 1),
    Guide.title("Proportion des modes de transport en fonction\nde la distance réseau jusqu'au centre-ville\nà partir du domicile du ménage"),
    Guide.xlabel("Distance réseau jusqu'au centre-ville\nà partir du domicile du ménage (km)"), Guide.ylabel("Proportion"),
    Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Du graphique de fréquence, nous déterminons que la majorité des gens questionnés ont un domicile proche de la Gare Centrale de Montréal. Ceci pourrait être dû au fait que la densité de population est plus élevée plus proche du centre-ville.
    
Nous voyons sur le graphique de proportion qu'il y a une corrélation inverse entre la distance du centre-ville et l'emprunt de modes de transport actifs ou collectifs, et une corrélation positive avec l'utilisation de la voiture ou du mode de transport autre. Nous observons également que le mode de transport hybride est le plus utilisé par les gens vivant à une distance d'entre 10 et 40 km du centre-ville de Montréal.

<a id="exploratory_analysis_sex"></a>
### Sexe de la personne 

Le sexe de la personne répondant, soit homme ou femme.

#### Fréquence et proportion des modes de transport en fonction du sexe de la personne 

In [22]:
df = histcat(train, :P_SEXE_str, :P_SEXE)
count_plot = plot(df, x = :P_SEXE_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction du sexe de la personne"), Guide.xlabel("Sexe"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_SEXE_str, :P_SEXE)
proportion_plot = plot(df, x = :P_SEXE_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction du sexe de la personne"), Guide.xlabel("Sexe"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Nous remarquons ici que les femmes sont moins susceptibles que les hommes de se déplacer en voiture, et privilégient un peu plus le transport collectif et le transport hybride. Les proportions de femmes et d'hommes utilisant le transport autre ou actif semblent être sensiblement les mêmes.

<a id="exploratory_analysis_age"></a>
### Âge de la personne 

Cette variable correspond à l'âge de la personne, entre 1 et 99 ans.

#### Fréquence et proportion des modes de transport en fonction de l'âge de la personne

In [23]:
bin_count = 100
x_min = 0
x_max = 100
count_plot = plot(train, x = :P_AGE, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :P_AGE)
proportion_plot = plot(df, x = :P_AGE, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de l'âge catégorisé de la personne

In [24]:
df = histcat(train, :P_AGE_str, :P_AGE)
count_plot = plot(df, x = :P_AGE_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_AGE_str, :P_AGE)
proportion_plot = plot(df, x = :P_AGE_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Une tendance très claire se dessine ici: les comportements des personnes de moins de 17 ans et de plus de 17 ans en termes de transport sont complètement différents. Nous pouvons en déduire qu'il serait peut-être préférable de séparer notre échantillon en deux afin de créer des modèles séparés pour ces deux groupes d'âge.  

Les moins de 17 ans constituent la grande majorité des utilisateurs du mode de transport autre (fort probablement les autobus scolaires), et utilisent en beaucoup plus grand nombre le transport actif. Ils se déplacent également moins en voiture que les personnes plus âgées. Nous observons aussi que les enfants de moins de 11 ans utilisent très rarement le transport collectif.  

Pour ce qui en est des adultes, il y a une corrélation claire entre l'âge et la proportion de personnes utilisant la voiture pour le déplacement. De plus, plus une personne est âgée, moins elle risque de prendre le transport collectif. Il n'y a pas de tendance claire quant aux modes de transports actifs et hybrides pour cette tranche d'âge.

<a id="exploratory_analysis_age_group"></a>
### Groupe d'âge de la personne

Les âges des personnes sont initialement regroupés en 11 catégories dans le jeu de données :

| Indice | Catégorie |
|---|---|
| 1 | 0 à 4 ans|
| 2 | 5 à 9 ans|
| 3 | 10 à 14 ans |
| 4 | 15 à 19 ans |
| 5 | 20 à 24 ans |
| 6 | 25 à 34 ans |
| 7 | 35 à 44 ans |
| 8 | 45 à 54 ans |
| 9 | 55 à 64 ans |
| 10 | 65 à 74 ans |
| 11 | 75 ans et plus |

Nous avons décidé de regrouper les âges en catégories différentes afin de proposer une division plus représentative des tendances des différents groupes d'âge :

| Indice | Catégorie |
|---|---|
| 1 | < 11 |
| 2 | [11, 17[ |
| 3 | [17, 20[ |
| 4 | [20, 25[ |
| 5 | [25, 30[ |
| 6 | [30, 35[ |
| 7 | [35, 40[ |
| 8 | [40, 45[ |
| 9 | [45, 50[ |
| 10 | [50, 55[ |
| 11 | [55, 60[ |
| 11 | [60, 65[ |
| 11 | [65, 70[ |
| 11 | >= 70 |

#### Fréquence et proportion des modes de transport en fonction du groupe d'âge de la personne

In [25]:
df = histcat(train, :P_GRAGE_str, :P_GRAGE)
count_plot = plot(df, x = :P_GRAGE_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_GRAGE_str, :P_GRAGE)
proportion_plot = plot(df, x = :P_GRAGE_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

In [26]:
df = histcat(train, :P_AGE_group_str, :P_AGE)
count_plot = plot(df, x = :P_AGE_group_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_AGE_group_str, :P_AGE)
proportion_plot = plot(df, x = :P_AGE_group_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Ces graphiques illustrent une tendance équivalente à celle observée avec la variable continue associée. Il est maintenant encore plus évident de percevoir la distinction entre les moins de 17 ans et les 17 ans et plus.

<a id="exploratory_analysis_occupation"></a>
### Occupation principale de la personne

Les occupations principales des personnes sont regroupées en 8 catégories :

| Indice |         Catégorie         |
|:------:|:-------------------------:|
|    1   | Travailleur à temps plein |
|    2   |  Travailler à temps plein |
|    3   |          Étudiant         |
|    4   |          Retraité         |
|    5   |           Autre           |
|    6   |  Enfant de 4 ans et moins |
|    7   |        À la maison        |
|    8   |           Refus           |

Nous avons jugé pertinent d'ajouter la variable explicative P_etudiant afin de simplement souligner si une personne est étudiante ou non :

| Indice |         Catégorie         |
|:------:|:-------------------------:|
|    1   | Étudiant |
|    2   |  Autre |

#### Fréquence et proportion des modes de transport en fonction de l'occupation principale de la personne

In [27]:
df = histcat(train, :P_STATUT_str, :P_STATUT)
count_plot = plot(df, x = :P_STATUT_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde l'occupation principale de la personne"), Guide.xlabel("Occupation principale de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_STATUT_str, :P_STATUT)
proportion_plot = plot(df, x = :P_STATUT_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde l'occupation principale de la personne"), Guide.xlabel("Occupation principale de la personne"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Fréquence et proportion des modes de transport en fonction du statut étudiant de la personne

In [28]:
df = histcat(train, :P_etudiant_str, :P_etudiant)
count_plot = plot(df, x = :P_etudiant_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu statut étudiant de la personne"), Guide.xlabel("Statut étudiant de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_etudiant_str, :P_etudiant)
proportion_plot  = plot(df, x = :P_etudiant_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu statut étudiant de la personne"), Guide.xlabel("Statut étudiant de la personne"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Dans le premier graphique de fréquence, nous voyons que la grande majorité des répondants sont soit des travailleurs à temps plein ou des étudiants. Ensuite, il y a une petite proportion de travailleurs à temps partiel, et une très petite portion de retraités, de personnes restant à la maison ou dans une situation d'emploi "autre". Nous observons également que les étudiants ont un comportement très différent en termes de transport par rapport aux travailleurs, En effet, il semble utiliser davantage le mode de transport autre (probablement l'autobus scolaire) et se déplace beaucoup moins en voiture.  

En séparant les occupations en deux catégories, étudiant et non-étudiant, nous décelons une tendance très forte : presque tous les utilisateurs du mode de transport autre sont étudiants. Les étudiants sont également environ 2.5 fois moins susceptibles d'utiliser la voiture pour se déplacer, et se déplacent environ trois fois plus de manière active.

<a id="exploratory_analysis_drivers_license"></a>
### Possession d’un permis de conduire de la personne

Cette variable indique si la personne est en possession d'un permis valide, si elle est éligible, si elle n'en a pas ou si elle refuse de répondre à la question. Nous jugé pertinent d'ajouter une variable modifiant les catégories initalement en seulement trois catégories distinctes, c'est-à-dire Oui, Non et Non applicable.

#### Fréquence et proportion des modes de transport en fonction de la possession d’un permis de conduire de la personne

In [29]:
df = histcat(train, :P_PERMIS_str, :P_PERMIS)
count_plot = plot(df, x = :P_PERMIS_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la possession d’un permis de conduire de la personne"), Guide.xlabel("Possession d’un permis de conduire de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_PERMIS_str, :P_PERMIS)
proportion_plot = plot(df, x = :P_PERMIS_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la possession d’un permis de conduire de la personne"), Guide.xlabel("Possession d’un permis de conduire de la personne"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Fréquence et proportion des modes de transport en fonction de la possession d’un permis de conduire de la personne

In [30]:
df = histcat(train, :P_PERMIS_modif_str, :P_PERMIS_modif)
count_plot = plot(df, x = :P_PERMIS_modif_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la possession d’un permis de conduire de la personne"), Guide.xlabel("Possession d’un permis de conduire de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :P_PERMIS_modif_str, :P_PERMIS_modif)
proportion_plot = plot(df, x = :P_PERMIS_modif_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde la possession d’un permis de conduire de la personne"), Guide.xlabel("Possession d’un permis de conduire de la personne"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Comme presque aucun répondant ne refusait de répondre à la question ou ne savait pas s'ils étaient en possession d'un permis de conduire, nous avons éliminé ces deux catégories.  
    
Comme nous pouvions nous y attendre, les personnes ayant un permis de conduire sont celles se déplaçant le plus en voiture. Les personnes éligibles pour un permis de conduire n'en ayant pas utilisent majoritairement le transport collectif, et se déplacent très peu en voiture. Ils utilisent également plus le transport actif. Les personnes non éligibles pour l'obtention d'un permis se déplacent moins en voiture que ceux en ayant un, mais plus que ceux éligibles n'en ayant pas. Ces personnes se déplacent le plus en mode de transport autre ou de manière active.

<a id="exploratory_analysis_departure_time"></a>
### Heure de départ du déplacement 

La variable explicative, définie entre 0 et 2800, correspond aux heuresminutes du début du déplacement. Les deux derniers chiffres correspondent aux minutes, tandis que ceux les précédant correspondent aux heures. Les heures de départ entre 2400 et 2800 correspondent à des déplacements de retour du travail après minuit. Nous avons décidé de catégoriser par heure les données afin de pouvoir mieux visualiser les graphiques.

#### Fréquence et proportion des modes de transport en fonction de l'heure de départ du déplacement 

In [31]:
bin_count = 100
x_min = 0
x_max = 2800
count_plot = plot(train, x = :D_HREDE, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction\nde l'heure de départ du déplacement"), Guide.xlabel("Heure de départ du déplacement"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :D_HREDE)
proportion_plot = plot(df, x = :D_HREDE, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\nde l'heure de départ du déplacement"), Guide.xlabel("Heure de départ du déplacement"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: train not defined

#### Fréquence et proportion des modes de transport en fonction de l'heure de départ catégorisée du déplacement 

In [32]:
df = histcat(train, :D_HREDE_str, :D_HREDE)
count_plot = plot(df, x = :D_HREDE_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde l'heure de départ du déplacement"), Guide.xlabel("Heure de départ du déplacement"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :D_HREDE_str, :D_HREDE)
proportion_plot = plot(df, x = :D_HREDE_str, y = :Proportion, color = :D_Mode_str, Geom.bar,Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction de l'heure de départ du déplacement"), Guide.xlabel("Heure de départ du déplacement"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion 

Nous remarquons ici que c'est très improbable pour les personnes partant entre une heure et quatre heures du matin d'utiliser le transport collectif. De plus, le laps de temps où la voiture est la moins utilisée est entre sept et quatorze heures, au profit de modes comme autre, transport collectif et actif. 

<a id="exploratory_analysis_traveling_reason"></a>
### Motif du déplacement

Le motif du déplacement correspond à la raison pour laquelle une personne se déplace. Elle contient 13 catégories :  

| Indice |         Catégorie         |
|:------:|:-------------------------:|
|    1   | Travail |
|    2   |  Rendez-vous d'affaires |
|    3   |          Sur la route         |
|    4   |          École         |
|    5   |           Magasinage           |
|    6   |  Loisir |
|    7   |        Visite d'ami(e)s/parenté        |
|    8   |           Santé           |  
|    9   |          Reconduire quelqu'un         |
|    10   |           Chercher quelqu'un           |
|    11   |  Retour au domicile |
|    12   |        Autre        |
|    13   |           Indéterminé/refus/NSP           | 

Les répondants n'ont saisi que les motifs Travail et École; nous avons donc décidé de laisser tomber toutes les autres catégories.

#### Fréquence et proportion des modes de transport en fonction du motif du déplacement

In [33]:
df = histcat(train, :D_MOTIF_str, :D_MOTIF )
count_plot = plot(df, x = :D_MOTIF_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu motif du déplacement"), Guide.xlabel("Motif du déplacement"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))
df = histnormcat(train, :D_MOTIF_str, :D_MOTIF)
proportion_plot = plot(df, x = :D_MOTIF_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu motif du déplacement"), Guide.xlabel("Motif du déplacement"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))
draw(SVGJS(30cm, 15cm), hstack([count_plot, proportion_plot]))

UndefVarError: UndefVarError: histcat not defined

#### Discussion

Seulement deux motifs ont été donnés pour les déplacements : le déplacement à l'école ou au travail. Les comportements sont très différents : les personnes se rendant au travail sont presque trois fois plus susceptibles d'utiliser la voiture et n'utilisent pas le mode de transport autre, tandis que les personnes se rendant à l'école sont plus susceptibles d'utiliser le transport autre (en l'occurrence, l'autobus scolaire), et le transport actif (environ trois fois plus probable que pour les travailleurs).

<a id="model_creation"></a>
## Création des modèles 

Les différents modèles conçus au cours de la compétition nous ont permis d'itérer sur notre solution de classification afin d'y établir notre solution la plus prédictive. La section qui suit illustre notre processus de création de modèles et de validation de modèles. Elle identifiera les techniques de validation et de choix de modèles, les différents types de modèles essayés, le processus détaillé de la conception de notre modèle final ainsi que les outils logiciels utilisés.

<a id="model_validation"></a>
### Validation des modèles

Afin de concevoir un modèle, nous avons initialement choisi les variables explicatives du modèle par la méthode essai-erreur jumelée à notre intuition de prédictibilité des variables explicatives analysées graphiquement lors de l'analyse exploratoire. Ainsi, pour chaque modèle établi, nous avons procédé par validation croisée pour évaluer son pouvoir prédictif en faisait une moyenne du pourcentage de déplacements correctement classés pour enfin ne considérer que le modèle ayant obtenu le pourcentage moyen le plus élevé.

Cependant, nous avons rapidement constaté que cette approche était fastidieuse et aléatoire, car le nombre de combinaisons possibles est immense et que bien que le choix des variables explicatives semble intuitif, il n'est pas nécessairement optimal. Nous avons donc opté pour une technique plus pragmatique.

En effet, pour chacun des types de modèles qui sera présenté dans la section [Essais](#model_tries), nous avons comparé le pourcentage de déplacements correctement classés de tous de modèles possibles, c'est-à-dire les modèles conçus à partir de toutes les combinaisons possibles de variables explicatives. De cette manière, nous avons pu être en mesure de sélectionner le modèle et les variables explicatives qui prédisent le mieux nos données. Bien que cette méthode soit couteuse en temps de calcul, elle nous a paru plus efficace en termes de temps par personne.

Voici un exemple en Julia de notre processus de validation et de choix des modèles :

```julia
using CSV, DataFrames, Gadfly, Distributions, StatsBase, MLBase, Random, Combinatorics, Core;
include("utils.jl");

train = CSV.read("ODtrain.csv")
clean_train_data!(train);

rand_train = train[shuffle(1:size(train, 1)),:];

feature_name_to_distribution = Dict(
     :D_MOTIF_cat => Categorical,
     :P_PERMIS_cat => Categorical,
     :M_AUTO_per_PERS_cat => Categorical,
     :M_DOMSM_cat => Categorical,
     :P_SEXE_cat => Categorical,
     :P_AGE_group_cat => Categorical,
     :D_HREDE_cat => Categorical,
     :P_etudiant_cat => Categorical,
     :M_emploiTC_cat => Categorical,
     :M_dist_centre_cat => Categorical,
     :M_dist_metro_cat => Categorical,
     :M_dist_train_cat => Categorical,
     :M_dist_tc_cat => Categorical,
     :P_STATUT_cat => Categorical,
     :P_GRAGE_cat => Categorical,
     :M_revenu_cat => Categorical,
     :M_AUTO_cat => Categorical,
)

file_name = "resultPredictions.txt"
if (isfile(file_name))
    throw(ErrorException(file_name * " already existing."))
end

results = DataFrame[]

nb_kfold = 10

n = size(train, 1)

for feature_names in combinations(collect(keys(feature_name_to_distribution)))

    features = DataFrame(
        Name = feature_names,
        Distribution = map(x -> feature_name_to_distribution[x], feature_names)
    )
    
    scores = cross_validate(
        indices -> estimate_model(rand_train, indices, features, [1, 2, 3, 4, 5]),
        (model, indices) -> evaluate_estimated_model(rand_train, indices, model, results),
        n, Kfold(n, nb_kfold)
    )
    
    open(file_name, "a") do io
       write(io, join(features[:Name], ", ") * ":\n" * string(mean_and_std(scores)) * "\n\n")
   end;
end
```
De cette façon, nous écrivons dans un fichier texte, l'ensemble des résultats obtenus avec les différents modèles testés. Nous pouvons ensuite faire le choix du modèle avec le meilleur pourcentage de prédiction. Il ne reste plus qu'à refaire le modèle avec les variables explicatives conclusives, de prédire les modes de transport de l'ensemble de test et de soumettre notre solution.

Il est à noter que des précisions sur la validation croisée ainsi que sur les outils utilisés pour effectuer le "k-fold" et la combinatoire sont fournies à la section [Outils](#model_utils).

<a id="model_tries"></a>
### Essais

Cette section présente les trois types de modèles que nous avons conçus avec des exemples de prédiction pour chacun d'entre eux. Pour chaque type, nous expliquerons notre réflexion qui nous a poussé à le développer, nous préciserons les variables utilisées et nous offrirons une analyse des résultats obtenus pour un exemple de modèle appartenant à ce type.

<a id="model_continuous_variables_only"></a>
#### Variables continues seulement

Le premier type de modèle que nous avons essayé se caractérise par l'utilisation exclusive des variables continues fournies dans le jeu de données. Nous avons choisi de commencer avec ce type de modèle dans le but d'évaluer la qualité de nos variables continues. Pour ce faire, au lieu de choisir une distribution aléatoirement ou instinctivement, pour chaque variable explicative continue, nous avons utilisé une méthode pragmatique. Ainsi, pour chaque distribution disponible dans la bibliothèque **Distributions** ainsi que pour chaque variable explicative continue, nous avons évalué le BIC du modèle produit par la variable explicative avec une certaine distribution. Ensuite, nous conservons la distribution correspondante au meilleur modèle selon le BIC.

Voici la fonction, que nous avons implémentée et qui est disponible dans le fichier *utils.jl*,  qui sert à calculer la meilleure distribution selon les BIC évalués pour chaque modèle :

``` Julia
function find_best_distribution(data::DataFrame, feature_name::Symbol)
    distributions = [Arcsine, Beta, BetaPrime, Biweight, Cauchy, Chi, Chisq, Cosine, Epanechnikov, Erlang, Exponential, FDist, Frechet, Gamma, GeneralizedExtremeValue, GeneralizedPareto, Gumbel, InverseGamma, InverseGaussian, Kolmogorov, KSDist, KSOneSided, Laplace, Levy, Logistic, LogNormal, NoncentralBeta, NoncentralChisq, NoncentralF, NoncentralT, Normal, NormalCanon, NormalInverseGaussian, Pareto, Rayleigh, Semicircle, StudentizedRange, SymTriangularDist, TDist, TriangularDist, Triweight, Uniform, VonMises, Weibull]
    best_bic = typemin(Float64)
    best_distribution = nothing
    for distribution in distributions
        features = DataFrame(
            Name = feature_name,
            Distribution = distribution
        )
        
        try
            model = train_model(data, features, [1, 2, 3, 4, 5])
            distributions_per_mode = model[feature_name]
            loglikelihood_per_mode = Float64[]
            k = 0
            for mode in 1:5       
                data_of_mode = filter(x -> x[:D_Mode] == mode, data)
                values_of_mode = data_of_mode[feature_name]
                k += length(params(distributions_per_mode[mode]))
                loglikelihood_of_mode = loglikelihood(distributions_per_mode[mode], values_of_mode)
                push!(loglikelihood_per_mode, loglikelihood_of_mode)
            end
            bic = sum(loglikelihood_per_mode) - k / 2 * log(size(data, 1))

            if bic > best_bic
                best_bic = bic
                best_distribution = distribution
            end
        catch
        end
    end
    
    return best_distribution
end;
```

Pour ce type de modèle, nous avons seulement considéré les variables continues du jeu de données, c'est-à-dire M_dist_train, M_dist_metro, M_emploiTC , M_dist_centre, P_AGE et D_HREDE. Voici les graphiques d'analyse des lois de chaque variable explicative continue :

In [34]:
bin_count = 100
x_min = 0
x_max = 65

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(LogNormal, filtered_data[:M_dist_train])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :M_dist_train, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Distance réseau (km)"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de la distance réseau de la station de train\nla plus proche à partir du domicile du ménage\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

In [35]:
bin_count = 100
x_min = 0
x_max = 65

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(Gamma, filtered_data[:M_dist_metro])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :M_dist_metro, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Distance réseau (km)"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de la distance réseau de la station de métro\nla plus proche à partir du domicile du ménage\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

In [36]:
bin_count = 200
x_min = 0
x_max = 200000

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(Exponential, filtered_data[:M_emploiTC])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :M_emploiTC, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Accessibilité du ménage à l’emploi en transport en commun\n (Nombre d'emplois)"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de l'accessibilité du ménage à l’emploi en transport en commun\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

In [37]:
bin_count = 100
x_min = 0
x_max = 65

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(Gamma, filtered_data[:M_dist_centre])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :M_dist_centre, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Distance réseau (km)"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de la distance réseau jusqu'au centre-ville à partir du domicile du ménage\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

In [38]:
bin_count = 100
x_min = 0
x_max = 100

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(Gamma, filtered_data[:P_AGE])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :P_AGE, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Âge"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de l'âge de la personne\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

In [39]:
bin_count = 100
x_min = 0
x_max = 2800

plots = Plot[]

modes = unique(train[:D_Mode])
for mode in modes
    filtered_data = filter(x -> x[:D_Mode] == mode, train)
    filtered_data_count = size(filtered_data, 1)
    nb_bin_count = floor(Int, sqrt(filtered_data_count))

    fd = fit(Cauchy, filtered_data[:D_HREDE])
    xx = range(x_min, length = nb_bin_count * 5, stop = x_max)

    h = layer(filtered_data, x = :D_HREDE, Geom.histogram(bincount = nb_bin_count, density = true), Theme(default_color = "deepskyblue"))
    d = layer(x = xx, y = pdf.(fd, xx), Geom.line, Theme(default_color = "red"))
    p = plot(d, h, Coord.cartesian(xmin = x_min, xmax = x_max),
        Guide.title("$(get_mode_string_category(mode))"),
        Guide.xlabel("Heure de départ du déplacement (heuresminutes)"), Guide.ylabel(""))
    push!(plots, p)
end;

grid = title(gridstack(Union{Plot,Compose.Context}[plots[1] plots[2]; plots[3] plots[4]; plots[5] Compose.context()]), "Fonction de densité de l'heure de départ du déplacement\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 30cm), grid)

UndefVarError: UndefVarError: train not defined

Voici un exemple fonctionnel d'une validation croisée à partir des meilleures distributions pour toutes les variables explicatives continues du problème :

In [40]:
rand_train = train[shuffle(1:size(train, 1)),:];

feature_name_to_distribution = Dict(
    :M_dist_train => LogNormal,
    :M_dist_metro => Gamma,
    :M_emploiTC => Exponential,
    :M_dist_centre => Gamma,
    :P_AGE => Gamma,
    :D_HREDE => Cauchy,
)
features = convert_feature_dict_to_features(feature_name_to_distribution)

results = DataFrame[]

nb_kfold = 5

n = size(train, 1)

scores = cross_validate(
    indices -> estimate_model(rand_train, indices, features, [1, 2, 3, 4, 5]),
    (model, indices) -> evaluate_estimated_model(rand_train, indices, model, results),
    n, Kfold(n, nb_kfold)
)

println("{" * join(features[:Name], ", ") * "}\n" * string(mean_and_std(scores)) * "\n")

UndefVarError: UndefVarError: train not defined

Nous obtenons un pourcentage de déplacements correctement classés évalué à environ 57%. Il est évident qu'il y a amélioration sur une prédiction basée sur la proportion de voitures au sein de l'ensemble d'entraînement (l'exemple de soumission à 54% sur Kaggle). Néanmoins, une progression de 3% est mince et comme il est possible de le constater visuellement avec les graphiques d'analyse des lois de chaque variable explicative continue, les lois ne suivent pas parfaitement les données. 

Afin d'être en mesure de mieux comprendre nos prédictions, voici la matrice de confusion obtenue à partir des résultats :

In [41]:
confusion_matrix = zeros(Int, 5, 5)
actuals = results[1][:D_Mode]
predictions = results[1][:Prediction]
confusion_matrix = confusmat(5, actuals, predictions)

predicted = ["Mode prédit \n $(get_mode_string_category(i))" for i = 1:5]
actual = ["Mode réel \n $(get_mode_string_category(i))" for i = 1:5]

succeeded_counts = [confusion_matrix[i,i] for i in 1:5]
sort!(succeeded_counts)

z = reshape([i == j ? findfirst(x -> x == confusion_matrix[i,j], succeeded_counts) : 0 for i in 1:5 for j in 1:5], 5, 5)
annotations= [(j - 0.5, i - 0.5, confusion_matrix[i, j]) for i = 1:5 for j = 1:5]
heatmap(predicted, actual, z, annotations = annotations, aspect_ratio = 1, c=ColorGradient(["#f5f5f5", "#005aa7"]), legend = :none, size = (650, 650))

UndefVarError: UndefVarError: results not defined

Globalement, nos pouvons conclure de cette matrice de confusion que nos prédictions sur les modes de transport comme Transport Collectif, Actif et Autre sont invalides dans la majorité des cas. Il est donc facile d'en déduire que l'utilisation d'une combinaison de variables continues n'est fort probablement pas une excellente idée. C'est la raison pour laquelle, c'est dans le but de comparer la puissance prédictive de nos variables discrètes que nous essayerons seulement les variables catégorielles dans le prochain type d'essai.

<a id="model_categorical_variables_only"></a>
#### Variables catégorielles seulement

Le second type de modèle que nous avons essayé se caractérise par l'utilisation exclusive des variables catégorielles fournies dans le jeu de données. Nous avons choisi ce type de modèle dans le but d'évaluer la qualité de nos variables catégorielles. Pour ce faire, nous avons utilisé le code détaillé plus haut qui évalue les différentes combinaisons de variables dans le but de trouver le modèle conçu par le meilleur ensemble de variables catégorielles. Voici un exemple fonctionnel d'une validation croisée à partir du meilleur ensemble de variables explicatives catégorielles du problème :

In [42]:
feature_name_to_distribution = Dict(
    :P_GRAGE => Categorical,
    :M_DOMSM_cat => Categorical,
    :P_SEXE => Categorical,
    :M_revenu => Categorical,
    :P_PERMIS => Categorical,
    :P_STATUT => Categorical,
)
features = convert_feature_dict_to_features(feature_name_to_distribution)

results = DataFrame[]

nb_kfold = 5

n = size(train, 1)

scores = cross_validate(
    indices -> estimate_model(rand_train, indices, features, [1, 2, 3, 4, 5]),
    (model, indices) -> evaluate_estimated_model(rand_train, indices, model, results),
    n, Kfold(n, nb_kfold)
)

println("{" * join(features[:Name], ", ") * "}\n" * string(mean_and_std(scores)) * "\n")

UndefVarError: UndefVarError: convert_feature_dict_to_features not defined

Nous obtenons un pourcentage de déplacements correctement classés évalué à environ 63%. Il y a donc une grande amélioration sur l'exemple de prédiction sur Kaggle (54%) ainsi que sur notre prédiction précédente (57%). Comme il semble que le pouvoir prédictif des variables catégorielles est élevé, nous sommes venus à la conclusion que la catégorisation ou la recatégorisation des variables pourraient être une solution envisageable et viable.

Afin d'être en mesure de mieux comprendre la qualité de nos prédictions, voici la matrice de confusion obtenue à partir des résultats :

In [43]:
confusion_matrix = zeros(Int, 5, 5)
actuals = results[1][:D_Mode]
predictions = results[1][:Prediction]
confusion_matrix = confusmat(5, actuals, predictions)

predicted = ["Mode prédit \n $(get_mode_string_category(i))" for i = 1:5]
actual = ["Mode réel \n $(get_mode_string_category(i))" for i = 1:5]

succeeded_counts = [confusion_matrix[i,i] for i in 1:5]
sort!(succeeded_counts)

z = reshape([i == j ? findfirst(x -> x == confusion_matrix[i,j], succeeded_counts) : 0 for i in 1:5 for j in 1:5], 5, 5)
annotations= [(j - 0.5, i - 0.5, confusion_matrix[i, j]) for i = 1:5 for j = 1:5]
heatmap(predicted, actual, z, annotations = annotations, aspect_ratio = 1, c=ColorGradient(["#f5f5f5", "#005aa7"]), legend = :none, size = (650, 650))

UndefVarError: UndefVarError: results not defined

À l'aide de la matrice de confusion, il est possible de voir une amélioration significative de la justesse de la prédiction des modes de transport comme Voiture et Transport Collectif. Cependant, tout comme avec les variables continues, nos prédictions sur le mode Actif ainsi que sur le mode Autre sont majoritaires erronées.

<a id="model_categorical_and_categorized_continuous_variables"></a>
#### Variables catégorielles et continues catégorisées

Le troisième type de modèle que nous avons essayé se caractérise par l'utilisation de chacun des variables du jeu de données. Par contre, pour ce faire, nous avons aussi ajouté de nouvelles variables créées à partir des variables existantes. Nous avons d'ailleurs transformé les variables continues en variables catégorielles et nous avons modifié certaines catégories des variables catégorielles déjà présentes. Le code derrière ces transformations et ajouts se retrouve dans le fichier Julia *utils.js* que nous avons implémenté. Ce sont notamment les fonctions **clean\_train\_data** ainsi que **clean\_test\_data** qui s'en occupent.

##### Création des nouvelles variables explicatives catégorielles
À la suite de la conception de nos deux premiers types de modèles, nous avons pris conscience de la puissance prédictive des variables catégorielles pour notre jeu de données. Donc, nous avons créé de nouvelles variables explicatives à partir des variables fournies dans le but d'exposer de nouvelles tendances. 

D'abord, nous avons créé la variable explicative [M_AUTO_per_PERS](#exploratory_analysis_nb_vehicles_per_person). Le graphique de sa fréquence et de sa proportion par mode ainsi que les détails sur nos choix de catégories sont disponibles dans l'analyse exploratoire.

Ensuite, nous avons créé la variable explicative [M_dist_tc](#exploratory_analysis_distance_tc) qui correspond à la distance réseau minimale du ménage à la station de métro ou de train la plus proche. Bien que de base, la variable est continue, nous avons aussi catégorisé ses données associées. Le graphique de sa fréquence et de sa proportion par mode ainsi que les détails sur nos choix de catégories sont disponibles dans l'analyse exploratoire.

Après, nous avons créé la variable explicative [P_etudiant](#exploratory_analysis_occupation) qui correspond aux statuts des personnes catégorisés en étudiant ou autre. Le choix de ces deux catégories vient de la forte distinction entre ces deux groupes. Le graphique de sa fréquence et de sa proportion par mode ainsi que les détails sur nos choix de catégories sont disponibles dans l'analyse exploratoire.

Finalement, nous avons créé la variable explicative [P_PERMIS_modif](#exploratory_analysis_occupation) qui correspond à une simplification de la variable explicative P_PERMIS. Nous avons donc éliminé les catégories Ne sait pas (3) et Refus (4) puisque presque aucun répondant ne refusait de répondre à la question ou ne savait pas s'ils étaient en possession d'un permis de conduire. Ainsi, par manque de données, nous avons trouvé plus pertinents de les inclure dans la catégorie Non (2). Le graphique de sa fréquence et de sa proportion par mode ainsi que les détails sur nos choix de catégories sont disponibles dans l'analyse exploratoire.

##### Transformation des variables explicatives continues en variables catégorielles
Comme nos résultats avec les variables continues laissent à désirer et qu'aucune loi de probabilités singulière ne semble parfaitement correspondre à nos données, nous avons opté pour la catégorisation des variables continues fournies.

Voici la liste des variables continues qui ont été catégorisées dans l'analyse exploratoire ainsi que les raisons pour lesquelles ces catégorisations ont été effectuées :
- [P_AGE_group](#exploratory_analysis_age_group)  
    - Bien que le jeu de données fournisse P_GRAGE qui catégorise les âges des personnes, nous voulions personnaliser la catégorisation dans le but d'en retirer davantage d'informations. C'est d'ailleurs grâce à cette analyse sur la division des données par rapport aux âges que nous en sommes venus à notre modèle final.
- [M_dist_train](#exploratory_analysis_train_distance)  
    - En analysant les données disponibles, nous avons choisi les séparations catégorielles suivantes : [1.0, 2.0[, [2.0,3.0[, [3.0, 4.0[, [4.0, 5.0[, [5.0, 7.5[, [7.5, 10.0[, [10.0, 15.0[, [15.0, 20.0[, [20.0, 30.0[, >= 30.0. De cette façon, nous isolons adéquatement par plusieurs petits groupes les ménages se situant à proximité et par quelques larges groupes les ménages se situant plus loin. La division s'explique par une distribution approximative de la fréquence par catégorie.
- [M_dist_metro](#exploratory_analysis_metro_distance)  
    - Les choix des catégories sont les mêmes que pour la variable M_dist_train et s'expliquent pour les mêmes raisons.
- [M_emploiTC](#exploratory_analysis_public_transport_job_accessibility)  
    - Pour cette catégorie, nous avons jugé pertinent de faire des catégories de tailles approximatives comportant une proportion des données sensiblement équivalente entre elles.
- [M_dist_centre](#exploratory_analysis_downtown_distance)  
    - En analysant les données disponibles, nous avons choisi les séparations catégorielles suivantes : < 5.0, [5.0, 10.0[, [10.0, 15.0[, [15.0, 20.0[, [20.0, 25.0[, [25.0, 30.0[, [30.0, 35.0[, [35.0, 40.0[, [40.0, 50.0[, >= 50.0. Pour cette variable, nous avons simplement essayé de bien répartir nos données parmi chacune des catégories puisqu’aucun ensemble précis des données ne paraissait se distinguer particulièrement.
- [D_HREDE](#exploratory_analysis_departure_time)  
    -  Pour ce qui est de l'heure, nous trouvons qu'il est plus facile et simple d'analyser les données par heure plutôt que par heuresminutes. De cette manière, nous pouvons mieux distinguer les tendances des modes de transport à travers ces heures.

##### Modification des catégories des variables explicatives catégorielles
- [M_AUTO](#exploratory_analysis_nb_vehicles)  
    - La modification de la catégorisation du nombre de véhicules par ménage se caractérise simplement par le regroupement de tous les ménages dont le nombre de voitures est supérieur ou égal à 4 voitures en la catégorie 4+. Cette modification est justifiée par le peu de données fournies pour les ménages possédant une grande quantité de voitures et par leur proportion semblable aux ménages de 4 voitures.
- [M_PERS](#exploratory_analysis_nb_people)  
    - La modification de la catégorisation du nombre de personnes par ménage se caractérise simplement par le regroupement de tous les ménages dont le nombre de personnes est supérieur ou égal à 7 personnes en la catégorie 7+. Cette modification est justifiée par le peu de données fournies pour les ménages possédant un immense nombre de personnes et par leur proportion semblable aux ménages de 7 personnes.

Voici un exemple fonctionnel d'une validation croisée à partir du meilleur ensemble de variables explicatives du problème :

In [44]:
feature_name_to_distribution = Dict(
    :D_MOTIF_cat => Categorical,
    :M_AUTO_per_PERS_cat => Categorical,
    :M_DOMSM_cat => Categorical,
    :P_SEXE_cat => Categorical,
    :P_AGE_group_cat => Categorical,
    :P_PERMIS_modif_cat => Categorical,
    :D_HREDE_cat => Categorical,
)
features = convert_feature_dict_to_features(feature_name_to_distribution)

results = DataFrame[]

nb_kfold = 5

n = size(train, 1)

scores = cross_validate(
    indices -> estimate_model(rand_train, indices, features, [1, 2, 3, 4, 5]),
    (model, indices) -> evaluate_estimated_model(rand_train, indices, model, results),
    n, Kfold(n, nb_kfold)
)

println("{" * join(features[:Name], ", ") * "}\n" * string(mean_and_std(scores)) * "\n")

UndefVarError: UndefVarError: convert_feature_dict_to_features not defined

Nous obtenons un pourcentage de déplacements correctement classés évalué à environ 66%. Il y a donc une grande amélioration sur l'exemple de prédiction sur Kaggle (54%) ainsi que sur nos prédictions précédentes (63% et 57%). Nos décisions semblent avoir porté fruit, puisque nos prédictions sont considérablement plus précises. Par contre, il reste fort probablement place à amélioration en analysant davantage les données.

Afin d'être en mesure de mieux comprendre la qualité de nos prédictions, voici la matrice de confusion obtenue à partir des résultats :

In [45]:
confusion_matrix = zeros(Int, 5, 5)
actuals = results[1][:D_Mode]
predictions = results[1][:Prediction]
confusion_matrix = confusmat(5, actuals, predictions)

predicted = ["Mode prédit \n $(get_mode_string_category(i))" for i = 1:5]
actual = ["Mode réel \n $(get_mode_string_category(i))" for i = 1:5]

succeeded_counts = [confusion_matrix[i,i] for i in 1:5]
sort!(succeeded_counts)

z = reshape([i == j ? findfirst(x -> x == confusion_matrix[i,j], succeeded_counts) : 0 for i in 1:5 for j in 1:5], 5, 5)
annotations= [(j - 0.5, i - 0.5, confusion_matrix[i, j]) for i = 1:5 for j = 1:5]
heatmap(predicted, actual, z, annotations = annotations, aspect_ratio = 1, c=ColorGradient(["#f5f5f5", "#005aa7"]), legend = :none, size = (650, 650))

UndefVarError: UndefVarError: results not defined

À l'aide de la matrice de confusion, il est possible de voir une amélioration significative de la justesse de la prédiction des modes de transport comme Voiture, Transport Collectif et même Actif. Cependant, tout comme avec les modèles ultérieurs, nos prédictions du mode Autre restent majoritairement invalides.

<a id="final_model"></a>
### Modèle final

Notre solution finale au problème de modélisation du mode de transport des individus provient des itérations de chacun des modèles et types de modèles mis au point précédemment. L'idée principale derrière notre modèle final réside par la segmentation du jeu de données au moyen d'une analyse pertinente des variables explicatives. La conception de ce modèle se divise par une analyse du problème en sous-problèmes, par la séparation du jeu de données ainsi que par une optimisation de la solution obtenue. Une démonstration complète des résultats finaux est disponible à la fin de cette section.

#### Analyse du problème en sous-problèmes

Comme il a été possible de le constater à la section [Essais](#model_tries) avec notre troisième type de modèle, nous prédisons beaucoup trop souvent le mode de transport Autre. Nous allons donc analyser les déplacements fautivement prédits comme mode Autre. Le tableau suivant illustre et identifie les données sur les variables dont la différence entre leurs valeurs moyennes dans l'ensemble d'entraînement et l'ensemble des prédictions manquées avec le mode Autre est significativement grande.

In [46]:
wrong = DataFrame(filter(x -> x[:D_Mode] != 5 && x[:Prediction] == 5, eachrow(results[1])))
average = DataFrame()
average[:Titre] = ["Moyenne de l'ensemble d'entrainement" , "Moyenne des prédictions du mode Autre échouées"]

for name in names(wrong) 
    if name == :D_Mode_cat
        break
    end
    (train_mean, train_std) = mean_and_std(train[name])
    (wrong_mean, wrong_std) = mean_and_std(wrong[name])
    if train_mean > wrong_mean 
        if  wrong_mean + wrong_std < train_mean
            average[name] = [train_mean, wrong_mean]
        end
    else
        if  wrong_mean - wrong_std > train_mean
            average[name] = [train_mean, wrong_mean]
        end
    end    
end

display(average)

UndefVarError: UndefVarError: results not defined

Parmi ces variables explicatives trouvées, ceux que nous considérons comme pertinentes à analyser sont P_AGE, P_STATUT, P_PERMIS ainsi que P_MOTIF. Nous rejetons P_RANG qui n'est pas significatif et P_GRAGE qui se retrouve encodé dans P_AGE.

Nous allons donc analyser les fréquences des modes de transport en fonction des variables sélectionnées avec le jeu de données des déplacements fautivement prédits comme mode Autre.

In [47]:
df = histcat(wrong, :P_AGE_group_str, :P_AGE)
age = plot(df, x = :P_AGE_group_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"))

df = histcat(wrong, :P_STATUT_str, :P_STATUT)
statut = plot(df, x = :P_STATUT_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde l'occupation principale de la personne"), Guide.xlabel("Occupation principale de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"))

df = histcat(wrong, :P_PERMIS_str, :P_PERMIS)
permis = plot(df, x = :P_PERMIS_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\nde la possession d’un permis de conduire de la personne"), Guide.xlabel("Possession d’un permis de conduire de la personne"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"))

df = histcat(wrong, :D_MOTIF_str, :D_MOTIF )
motif = plot(df, x = :D_MOTIF_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu motif du déplacement"), Guide.xlabel("Motif du déplacement"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"))

grid = title(gridstack([age statut;permis motif]), "Fréquence des modes de transport des déplacements fautivement classifiés comme Autre\nselon les variables explicatives analysées\n", Compose.fontsize(18pt))
draw(SVGJS(30cm, 20cm), grid)

UndefVarError: UndefVarError: histcat not defined

Comme les précédents graphiques le démontrent, lorsque notre ancien modèle prédit fautivement le mode de transport Autre, les déplacements ne semblent provenir que d'enfants et adolescents étudiants n'ayant pas de permis de conduire et se déplaçant exclusivement à l'école. Comme ce segment de données s'applique à un sous-ensemble de déplacements très précis, il nous porte alors à croire que les enfants, les adolescents et les adultes ne se comporteraient pas tout à fait de la même façon et qu'on tirerait peut-être avantage à séparer notre problème en sous-problèmes chacun modélisable indépendamment.

Selon cette hypothèse, nous avons analysé de nouveau les fréquences des modes de transport en fonction de l'âge des personnes effectuant le déplacement.

In [48]:
bin_count = 100
x_min = 0
x_max = 100

f1 = plot(train, x = :P_AGE, color = :D_Mode_str, Geom.histogram(bincount = bin_count), Coord.cartesian(xmin = x_min, xmax = x_max), Guide.title("Fréquence des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))

df = histnorm(train, range(x_min, length = bin_count, stop = x_max), :P_AGE)
p1 = plot(df, x = :P_AGE, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(xmin = x_min, xmax = x_max, ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction de l'âge de la personne"), Guide.xlabel("Âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))

df = histcat(train, :P_AGE_group_str, :P_AGE)
f2 = plot(df, x = :P_AGE_group_str, y = :Count, color = :D_Mode_str, Geom.bar, Guide.title("Fréquence des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Fréquence"), Guide.colorkey(title = "Mode de transport"), Theme(dark_theme_without_colorkey))

df = histnormcat(train, :P_AGE_group_str, :P_AGE)
p2 = plot(df, x = :P_AGE_group_str, y = :Proportion, color = :D_Mode_str, Geom.bar, Coord.cartesian(ymin = 0, ymax = 1), Guide.title("Proportion des modes de transport en fonction\ndu groupe d'âge de la personne"), Guide.xlabel("Groupe d'âge"), Guide.ylabel("Proportion"), Guide.colorkey(title = "Mode de transport"))


grid = gridstack([f1 p1;f2 p2])
draw(SVGJS(30cm, 20cm), grid)

UndefVarError: UndefVarError: train not defined

Visuellement, les graphiques ci-dessus illustrent trois tendances bien définies. D'une part, la distinction entre les moins de 17 ans et les plus de 17 ans est assez drastique. Effectivement, très peu de déplacements avec le mode de transport Autre ne sont effectués par les plus de 17 ans. De plus, ce sont les moins de 17 ans qui caractérisent la grande majorité des déplacements actifs. Ainsi, il semble approprié d'au moins séparer les données par enfants et par adultes. Néanmoins, en portant attention aux groupes d'âge chez les moins de 17 ans, une autre grande distinction fait surface. Cela va sans dire que les moins de 11 ans n'utilisent presque pas le transport collectif comparativement aux 11 ans et plus. Donc, il semble pertinent d'effectuer une séparation des données en trois catégories distinctes séparées par l'âge des personnes effectuant les déplacements.

| Age | Nom |
| ------ | ------ |
| < 11 | Enfants |
| [11, 17[ | Adolescents |
| >= 17 | Adultes |

#### Séparation de l'ensemble de données

Comme établi précédemment, nous devons maintenant séparé notre ensemble de données par enfants, adolescents et adultes.

In [49]:
teenager_age_separator = 11
adult_age_separator = 17

train_kid = DataFrame(filter(x -> x[:P_AGE] < teenager_age_separator, train));
train_teenager = DataFrame(filter(x -> x[:P_AGE] >= teenager_age_separator && x[:P_AGE] < adult_age_separator, train));
train_adult = DataFrame(filter(x -> x[:P_AGE] >= adult_age_separator, train));

train_plot = plot(train, x = :D_Mode_str, Geom.histogram, Guide.title("Nombre de personnes employant\nchaque mode de transport"), Guide.xlabel("Mode de transport"), Guide.ylabel("Nombre de personnes"))
kid_plot = plot(train_kid, x = :D_Mode_str, Geom.histogram, Guide.title("Nombre d'enfants employant\nchaque mode de transport"), Guide.xlabel("Mode de transport"), Guide.ylabel("Nombre de personnes"))
teenager_plot = plot(train_teenager, x = :D_Mode_str, Geom.histogram, Guide.title("Nombre d'adolescent employant\nchaque mode de transport"), Guide.xlabel("Mode de transport"), Guide.ylabel("Nombre de personnes"))
adult_plot = plot(train_adult, x = :D_Mode_str, Geom.histogram, Guide.title("Nombre d'adultes employant\nchaque mode de transport"), Guide.xlabel("Mode de transport"), Guide.ylabel("Nombre de personnes"))

draw(SVGJS(30cm, 20cm), gridstack([train_plot kid_plot; teenager_plot adult_plot]))

UndefVarError: UndefVarError: train not defined

À l'aide de ces graphiques, il est facile de confirmer que chacun des sous-ensembles créés possède une tendance des modes de transport bien distincte. D'ailleurs, bien que nous réduisons la taille de notre jeu de données initial, chacun des nouveaux jeux de données engendrés par la segmentation du jeu de données initial comporte un nombre de données assez élevé pour être suffisant pour la classification bayésienne naïve.

#### Optimisation de la solution

Afin d'optimiser notre modèle, comme expliqué à la sous-section [Validation des modèles](#model_validation), nous avons sélectionné, pour chaque jeu de données, les variables explicatives résultantes au modèle dont le pourcentage de déplacements correctement prédits est le plus élevé par validation croisée.

En analysant nos divers essais avec une matrice de confusion sur une solution basée sur la séparation de l'ensemble de données, nous avons constaté que certains types de prédictions ne valaient même pas la peine d'être effectués. Ainsi, en appliquant un filtre sur la possibilité de prédiction des déplacements pour certains modes seulement, les modèles associés à un jeu de données sont alors plus performants. Pour les enfants, nous ne prédisons que pour les modes Voiture, Actif et Autre. Pour les adolescents, ce sont les modes Voiture, Transport Collectif, Actif et Autre qui sont prédits. Pour les adultes, Voiture et Transport Collectif sont les seules prédictions possibles. Ces choix ont été faits dans le but d'empêcher des tentatives de prédictions sur des modes dont la fréquence est mince et d'ainsi augmenter la probabilité de prédire un mode plus populaire.

#### Démonstration

Voici ci-bas la démonstration fonctionnelle de notre meilleur modèle. Elle comprend une validation croisée et une soumission des prédictions de l'ensemble de test à partir du meilleur ensemble de variables explicatives pour chacun des modèles conçus par segment du jeu de données initial :

##### Validation croisée

In [50]:
rand_train_kid = train_kid[shuffle(1:size(train_kid, 1)),:];
rand_train_teenager = train_teenager[shuffle(1:size(train_teenager, 1)),:];
rand_train_adult = train_adult[shuffle(1:size(train_adult, 1)),:];

nb_kfold = 10;

UndefVarError: UndefVarError: train_kid not defined

*Enfant*

In [51]:
feature_name_to_distribution_kid = Dict(
    :D_MOTIF_cat => Categorical,
    :M_AUTO_per_PERS_cat => Categorical,
    :M_DOMSM_cat => Categorical,
    :P_SEXE_cat => Categorical,
    :P_AGE_cat => Categorical,
    :D_HREDE_cat => Categorical
)
kid_features = convert_feature_dict_to_features(feature_name_to_distribution_kid)

kid_results = DataFrame[]

nb_kids = size(train_kid, 1)

modes_to_predict_kid = [1, 4, 5]

kid_scores = cross_validate(
    indices -> estimate_model(rand_train_kid, indices, kid_features, modes_to_predict_kid),
    (model, indices) -> evaluate_estimated_model(rand_train_kid, indices, model, kid_results),
    nb_kids, Kfold(nb_kids, nb_kfold)
)

println("{" * join(kid_features[:Name], ", ") * "}\n" * string(mean_and_std(kid_scores)) * "\n")

UndefVarError: UndefVarError: convert_feature_dict_to_features not defined

*Adolescent*

In [52]:
feature_name_to_distribution_teenager = Dict(
    :D_MOTIF_cat => Categorical,
    :M_AUTO_per_PERS_cat => Categorical,
    :M_DOMSM_cat => Categorical,
    :P_SEXE_cat => Categorical,
    :P_AGE_cat => Categorical,
    :D_HREDE_cat => Categorical
)
teenager_features = convert_feature_dict_to_features(feature_name_to_distribution_teenager)

teenager_results = DataFrame[]

nb_teenagers = size(train_teenager, 1)

modes_to_predict_teenager = [1, 2, 4, 5]

teenager_scores = cross_validate(
    indices -> estimate_model(rand_train_teenager, indices, teenager_features, modes_to_predict_teenager),
    (model, indices) -> evaluate_estimated_model(rand_train_teenager, indices, model, teenager_results),
    nb_teenagers, Kfold(nb_teenagers, nb_kfold)
)

println("{" * join(teenager_features[:Name], ", ") * "}\n" * string(mean_and_std(teenager_scores)) * "\n")

UndefVarError: UndefVarError: convert_feature_dict_to_features not defined

*Adulte*

In [53]:
feature_name_to_distribution_adult = Dict(
    :D_MOTIF_cat => Categorical,
    :M_AUTO_per_PERS_cat => Categorical,
    :M_DOMSM_cat => Categorical,
    :P_SEXE_cat => Categorical,
    :P_PERMIS_modif_cat => Categorical,
    :D_HREDE_cat => Categorical,
    :P_AGE_group_cat => Categorical
)
adult_features = convert_feature_dict_to_features(feature_name_to_distribution_adult)

adult_results = DataFrame[]

nb_kfold = 10

nb_adults = size(train_adult, 1)

modes_to_predict_adult = [1, 2]

adult_scores = cross_validate(
    indices -> estimate_model(rand_train_adult, indices, adult_features, modes_to_predict_adult),
    (model, indices) -> evaluate_estimated_model(rand_train_adult, indices, model, adult_results),
    nb_adults, Kfold(nb_adults, nb_kfold)
)

println("{" * join(adult_features[:Name], ", ") * "}\n" * string(mean_and_std(adult_scores)) * "\n")

UndefVarError: UndefVarError: convert_feature_dict_to_features not defined

*Total*

In [54]:
mean_and_std((kid_scores .* nb_kids .+ teenager_scores .* nb_teenagers .+ adult_scores .* nb_adults) ./ n)

UndefVarError: UndefVarError: kid_scores not defined

Nous obtenons un pourcentage de déplacements correctement classés évalué à environ 69%. Il y a donc une grande amélioration sur l'exemple de prédiction sur Kaggle (54%) ainsi que sur nos prédictions précédentes (66%, 63% et 57%). Notre décision de séparation du jeu de données semble être validée par l'augmentation de notre pourcentage de déplacements correctement prédits.

Afin d'être en mesure de mieux comprendre la qualité de nos prédictions, voici la matrice de confusion obtenue à partir des résultats :

In [55]:
results = deepcopy(adult_results[1])
append!(results, teenager_results[1])
append!(results, kid_results[1])

confusion_matrix = zeros(Int, 5, 5)
actuals = results[:D_Mode]
predictions = results[:Prediction]
confusion_matrix = confusmat(5, actuals, predictions)

predicted = ["Mode prédit \n $(get_mode_string_category(i))" for i = 1:5]
actual = ["Mode réel \n $(get_mode_string_category(i))" for i = 1:5]

succeeded_counts = [confusion_matrix[i,i] for i in 1:5]
sort!(succeeded_counts)

z = reshape([i == j ? findfirst(x -> x == confusion_matrix[i,j], succeeded_counts) : 0 for i in 1:5 for j in 1:5], 5, 5)
annotations= [(j - 0.5, i - 0.5, confusion_matrix[i, j]) for i = 1:5 for j = 1:5]
heatmap(predicted, actual, z, annotations = annotations, aspect_ratio = 1, c=ColorGradient(["#f5f5f5", "#005aa7"]), legend = :none, size = (650, 650))

UndefVarError: UndefVarError: adult_results not defined

À l'aide de la matrice de confusion, il est possible de voir une nette amélioration de la précision des prédictions de l'ensemble des modes de transport du problème. Avec la segmentation du jeu de données par âge, nous avons vu juste et ainsi réussi à améliorer les prédictions associées au mode Autre.

##### Soumission

In [56]:
test = CSV.read("ODtest.csv")
clean_test_data!(test);

ArgumentError: ArgumentError: "ODtest.csv" is not a valid file

In [57]:
kid_features = DataFrame(
    Name = [:D_MOTIF_cat, :M_AUTO_per_PERS_cat, :M_DOMSM_cat, :P_SEXE_cat, :P_AGE_cat, :D_HREDE_cat],
    Distribution = [Categorical, Categorical, Categorical, Categorical, Categorical, Categorical]
)

teenager_features = DataFrame(
    Name = [:D_MOTIF_cat, :M_AUTO_per_PERS_cat, :M_DOMSM_cat, :P_SEXE_cat, :P_AGE_cat, :D_HREDE_cat],
    Distribution = [Categorical, Categorical, Categorical, Categorical, Categorical, Categorical]
)

adult_features = DataFrame(
    Name = [:D_MOTIF_cat, :M_AUTO_per_PERS_cat, :M_DOMSM_cat, :P_SEXE_cat, :P_PERMIS_modif_cat, :D_HREDE_cat, :P_AGE_group_cat],
    Distribution = [Categorical, Categorical, Categorical, Categorical, Categorical, Categorical, Categorical]
)

model_kid = train_model(train_kid, kid_features, [1, 4, 5])
model_teenager = train_model(train_teenager, teenager_features, [1, 2, 4, 5])
model_adult = train_model(train_adult, adult_features, [1, 2])

predictions = Int64[]
for test_example in eachrow(test)
    if test_example[:P_AGE] < teenager_age_separator
        prediction = predict(model_kid, test_example) 
    elseif test_example[:P_AGE] >= teenager_age_separator && test_example[:P_AGE] < adult_age_separator
        prediction = predict(model_teenager, test_example) 
    else
        prediction = predict(model_adult, test_example)
    end
    push!(predictions, prediction)
end

output = DataFrame(IPERE = test[:IPERE], D_Mode = predictions);
CSV.write("soumission_equipe_4.csv", output)

UndefVarError: UndefVarError: train_model not defined

<a id="model_utils"></a>
### Outils

Cette sous-section présente les outils employés qui nous ont simplifié la tâche en termes de conception d'algorithme. Les éléments théoriques de ces outils sont expliqués afin de justifier leur utilisation.

#### MLBase (Cross Validation, k-fold)

Nous avons décidé d'utiliser la validation croisée "k-fold" à l'aide de la bibliothèque logicielle MLBase. La première étape de cette méthode est de diviser l'ensemble d'entraînement en k différents sous-ensembles de taille identiques. Par la suite, nous sélectionnons un des sous-ensembles comme ensemble de test, nous entraînons le modèle sur les k-1 sous-ensembles restants, puis nous calculons l'erreur quadratique de la prédiction sur l'ensemble k. Nous répétons cette procédure en prenant tour à tour les k-1 sous-ensembles restants comme ensembles de test. Finalement, nous faisons la moyenne des erreurs quadratiques des k estimations afin d'obtenir le pouvoir prédictif de notre modèle.


| Ensembles   | 1     | 2     | 3     | 4     | 5     |
|-------------|-------|-------|-------|-------|-------|
| Itération 1 | <font color="green">Train | <font color="green">Train | <font color="green">Train | <font color="green">Train | <font color="red">Test  |
| Itération 2 | <font color="green">Train | <font color="green">Train | <font color="green">Train |  <font color="red">Test | <font color="green">Train |
| Itération 3 | <font color="green">Train | <font color="green">Train | <font color="red">Test  | <font color="green">Train | <font color="green">Train |
| Itération 4 | <font color="green">Train | <font color="red">Test  | <font color="green">Train | <font color="green">Train | <font color="green">Train |
| Itération 5 | <font color="red">Test  | <font color="green">Train | <font color="green">Train | <font color="green">Train | <font color="green">Train |


Nous avons préféré la méthode "k-fold" de validation croisée à la méthode "LOOCV (Leave One Out Cross Validation)", qui est le cas particulier de la validation "k-fold" où on prend k=n. Cette décision a été prise, car avec la quantité de données en entrée, la méthode "LOOCV" aurait été extrêmement coûteuse temporellement, entravant la possibilité d'itérer à une vitesse convenable différents essais de configuration de nos modèles.

#### Combinatorics

La bibliothèque logicielle Combinatorics est utilisée afin de générer toutes les combinaisons de variables explicatives possibles, pour ensuite pouvoir tester ces combinaisons avec la validation croisée. Nous pouvons ainsi obtenir des statistiques quantitatives nous informant sur le potentiel de ces diverses combinaisons, guidant ainsi nos choix de variables pour les modèles.

#### Fonctions utilitaires

La plupart des fonctions que nous utilisons lors de la conception de nos modèles sont en fait des fonctions que nous avons définies dans *utils.jl*. 

##### Transformation de catégories

Comme expliqué à la section [Essais](#model_tries), nous avons créé de nouvelles variables explicatives et transformé celles déjà existantes. Ainsi, ces fonctions sont utilisées lors du traitement des données pour maximiser l'information explicative. Deux types de fonctions ont été conçues à cette fin.

La première suit la forme **get\_\*\_category** et retourne un entier entre 1 et le nombre de catégories voulues.

La seconde, quant à elle, suit la forme **get\_\*\_string\_category** et retourne une chaine de caractères associée à chaque catégorie d'une variable et utilisée pour rendre les graphiques plus informatifs.

##### Ajout de catégories

Afin de créer les nouvelles catégories, des appels aux fonctions **add\_category!** et **add\_string_category!** sont faits en passant les données existantes ou transformées pour ensuite les faire correspondre à des catégories afin de les ajouter au *DataFrame*. Cela permet de centraliser toutes nos données, et de simplement accéder aux nouvelles catégories par le *DataFrame* lors de la construction des graphiques et de l'entraînement de nos modèles prédictifs.

##### Traitement des données

Initialement, nous effectuons un appel à la fonction **clean\_train_data!**. Cette méthode fait des appels aux fonctions décrites ci-haut afin d'enlever les entrées manquantes avec la fonction **dropmissing!** et ensuite transforme les données initiales pour former les catégories et leurs descriptions qui nous intéressent. 

##### Affichage des données

Pour nous aider à détecter des tendances et la pertinence des différentes données à notre disposition, nous avons fait usage de graphiques pouvant afficher différentes statistiques concernant nos données. Pour ce faire, nous avons développé deux fonctions et en avons réutilisé une fournie par M. Jalbert.

La première, qui nous était fournie, est **histnorm** et permet d'obtenir les données nécessaires à la construction d'un histogramme de la proportion du nombre de déplacements pour chacun des modes pour une variable explicative continue donnée.

La seconde, **histnormcat**, est une variation de **histnorm**, mais permet d'obtenir les données nécessaires à la construction d'un histogramme de la proportion du nombre de déplacements pour chacun des modes pour une variable explicative catégorielle donnée.

La troisième, quant à elle, est **histcat** et permet d'obtenir les données nécessaires à la construction d'un histogramme du nombre de déplacements pour chacun des modes pour une variable explicative donnée.

##### Entraînement

Pour entraîner nos différents modèles, nous avons développé une fonction, **train\_model** pouvant entraîner, selon la méthode bayesienne naïve, un modèle avec un ensemble de variables explicatives et pour des modes de transport donnés. Ce dernier paramètre a été ajouté au fil des itérations de nos modèles lorsque, suite à une analyse des données, nous avons remarqué que pour certaines catégories de personnes, certains modes de transports ne sont pratiquement pas utilisés, ce qui nous permet de les omettre dans nos prédictions.

##### Prédiction

Une fois un modèle entraîné, nous pouvons l'exploiter en faisant appel à la méthode **predict** prenant un modèle ainsi qu'une entrée de notre jeu de donnée. Cette fonction calcule donc les log-probabilités des différents modes en fonction du modèle et des entrées et retourne le mode de transport ayant la plus haute probabilité selon les données fournies.

##### Validation croisée

Dû au nombre de soumissions limité sur Kaggle, nous avons utilisé la méthode de validation croisée "k-fold" pour évaluer nos différents modèles, comme il a été mentionné plus haut. Pour ce faire, nous avons défini deux fonctions, une pour estimer un modèle, et l'autre pour évaluer ledit modèle estimé.

La première méthode **estimate\_model** nous permet donc, à l'aide de la méthode **train\_model** étaillée plus haut, d'entraîner un modèle sur une partie de l'ensemble d'entraînement reçu en paramètre, toujours selon un ensemble de variables explicatives et des modes de transport reçus en paramètre. Nous nous trouvons donc à estimer ce modèle puisque seule une partie de l'ensemble d'entraînement est utilisée. Le modèle résultant est donc semblable à celui que l'on obtiendrait en appelant **train\_model**, mais il n'est pas aussi raffiné. 

La seconde, **evaluate\_estimated\_model**, évalue le modèle estimé en prédisant le mode de transport, pour chacune des entrées de test définies par la validation croisée "k-fold", et rapporte le résultat obtenu par le modèle, sous forme de pourcentage de réussite.  

##### Analyse de distributions

Enfin, lors de notre analyse exploratoire des données, nous avons développé et utilisé la méthode **find\_best\_distribution**, qui permet de trouver la distribution ayant le meilleur BIC pour une variable explicative donnée parmi une liste de distributions données.

<a id="possible_ameliorations"></a>
## Améliorations possibles

Outre nos multiples essais détaillés précédemment, il est certain que plusieurs autres améliorations auraient pu être mises en place dans le but d’augmenter nos prédictions sur l’ensemble de tests. Cette section vise donc à exposer les différentes techniques qui auraient été envisageables de faire afin d’améliorer le pouvoir prédictif de notre modèle final.


<a id="possible_ameliorations_feature_engineering"></a>
###  *Feature engineering*

Tout d’abord, avec l’approche bayésienne naïve, tout comme la majorité des approches en apprentissage machine, il est nécessaire d’utiliser les meilleures variables explicatives possibles afin de permettre à notre modèle de bien apprendre de nos données et d’être capable de bien prédire. De ce fait, il aurait été possible de perfectionner les variables explicatives utilisées à l’aide du *feature engineering*. Il est à noter que nous avons tenté de simples méthodes de *feature engineering* pour les variables qui nous semblaient les plus prometteuses, mais qu’une des améliorations possibles serait de tester avec toutes les variables. De plus, le processus de *feature engineering* pourrait être amélioré avec une meilleure connaissance du domaine. En effet, nous avons été en mesure d’exécuter quelques fois le processus de *feature engineering* complet qui consiste à faire une séance de remue-méninges ainsi qu’à faire le test des variables explicatives, à décider quelles variables créer ou modifier, à appliquer les modifications aux variables nécessaires, à vérifier le rôle de ces variables sur notre modèle actuel, à mettre au point les nouvelles variables afin d’améliorer le modèle, puis finalement de revenir à l’étape de remue-méninges afin de réappliquer ce processus. Cependant, plusieurs itérations de ce processus demandent une fine connaissance en analyse de données ainsi qu’un temps énorme. Il est certain qu’avec plus de temps, nous aurions pu faire un nombre plus élevé d’itérations, nous permettant de raffiner nos variables explicatives et par le fait même, notre modèle de prédiction.

<a id="possible_ameliorations_mixture"></a>
### Mélanges de lois

Ensuite, nous aurions pu opter pour l’utilisation de mélanges de lois plutôt que de discrétiser les variables explicatives continues. En effet, l’utilisation de tels mélanges nous aurait possiblement permis d’augmenter le pouvoir prédictif de notre modèle tout en conservant la richesse des données continues. De toute évidence, la discrétisation des données continues engendre une perte d’information qui aurait pu être conservée à des fins de prédictions. Néanmoins, un tel processus aurait demandé un temps d’analyse considérable, sans compter les connaissances nécessaires afin trouver les lois prédictives adéquates pour les données à estimer.

<a id="possible_ameliorations_different_distributions"></a>
### Distributions diverses

Puis, il aurait aussi été possible d’opter pour des distributions différentes par mode par variable afin d’augmenter le pouvoir prédictif du modèle. Bien entendu, nous avons tenté d’utiliser une seule loi de probabilités pour l’ensemble des modes de transports alors qu’une distribution distincte pour chacun des modes aurait été à essayer. En d’autres mots, nous utilisons le BIC afin de découvrir la loi maximisant la prédiction des cinq modes de transports d’une certaine variable explicative. Cependant, il aurait été pertinent d’utiliser une loi unique par mode de transport afin de réellement maximiser le pouvoir prédictif d’une loi par rapport au mode de transport de cette variable.

<a id="possible_ameliorations_new_solutions"></a>
### Autres techniques

Finalement, il est certain que nous aurions pu opter pour des techniques d'apprentissage machine différentes de celles vues en classe telle que l’apprentissage par réseaux de neurones. Malgré leur fort taux de popularité, ces techniques sont encore nébuleuses et demandent une compréhension dépassant largement les apprentissages faits lors du cours. De plus, les modèles générés par de telles techniques restent souvent inexpliqués dus à la complexité de leur création. Cela nous aurait défavorisés à cause de l’importance des points alloués à l’explication de notre modèle. Néanmoins, avec plus de temps, il aurait été pertinent de voir les résultats possibles avec l’utilisation de cadriciels tiers tels que *flux.jl* ou *TensorFlow.jl*.